# Indicators i1 & i2 of IRD Tuna Atlas

This notebook is available on https://github.com/juldebar/IRDTunaAtlas/blob/master/jupyter_notebook/sardara_notebook_i1i2.ipynb 

This notebook has been written and executed on https://blue-cloud-jupyterhub.d4science.org/ 

It uses data provided by Fishereries Atlas VRE

It can be used as a documentation of underlying processes used by the related shiny app https://github.com/juldebar/IRDTunaAtlas/blob/master/shiny/sardara_shiny_i1i2.Rmd

## Load R packages

In [ ]:
library(DBI)
library(sp)
library(sf)
library(plyr)
library(dplyr)
library(ggplot2)
library(plotly)

## 1. Connect the database

In [2]:
#######################
##Database Identifiers: tunaatlas / db-tunaatlas
source(file = "credentials.R")
con <- dbConnect(drv=DRV,  dbname = Dbname, host = Host , user=User, password=Password,port="5432")
source("https://raw.githubusercontent.com/juldebar/IRDTunaAtlas/master/R/TunaAtlas_i4_SpeciesMonthByOcean.R")

## 2. Execute SQL query to get data

In [ ]:
new_wkt <- 'POLYGON((-180 -90, 180 -90, 180 90, -180 90, -180 -90))'
wkt <- new_wkt
target_species <- c("YFT")
target_ocean <- c("ICCAT","IATTC","IOTC", "CCSBT")
target_gear <- c("PS","BB")
target_year <-  c("2000","2001","2002")

query_i4 <- "select species, year, month, ocean, value from public.i4 ;"
df_i4 <- st_read(con, query=query_i4)
species = unique(df_i4$species)
year = unique(df_i4$year)

## 3. Plot data with indicator 4

In [ ]:
if(wkt!=new_wkt){
  df_i4 = st_read(con, query =  paste0("SELECT species, ocean, year, month, sum(value) as value, unit, st_collect(geom) FROM public.i4_spatial WHERE ST_Within(geom,ST_GeomFromText('",wkt,"',4326)) GROUP BY species, ocean, year, month, unit ;"))
}

#Filtering input dataset with selected parameters
df_i4_filtered <- df_i4 %>% filter(year %in% target_year, species %in% target_species) 

i4 <- Atlas_i4_SpeciesMonthByOcean(df=as.data.frame(df_i4_filtered), 
                                          oceanAttributeName="ocean", 
                                          yearAttributeName="year", 
                                          monthAttributeName="month",
                                          speciesAttributeName="species", 
                                          valueAttributeName="value",
                                   withSparql=FALSE
                                      )
i4
ggplotly(i4)